In [1]:
import pandas as pd
from spswarehouse.warehouse import Warehouse
from spswarehouse.table_utils import *
from spswarehouse.googlesheets import GoogleSheets

To access your Google spreadsheet data, share the spreadsheet with spswarehouse@spswarehouse.iam.gserviceaccount.com


In [2]:
# This is the example from https://developers.google.com/sheets/api/quickstart/python
# Replace it with your own spreadsheet
spreadsheet_id = '1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms'
spreadsheet = GoogleSheets.open_by_key(spreadsheet_id)

# GoogleSheet
sheet = spreadsheet.sheet1

# Sheet as a DataFrame
df = pd.DataFrame(sheet.get_all_records())

# As a CSV file
csv_filename = '~/Desktop/tmp.csv'
df.to_csv(csv_filename, index=False)

In [3]:
# Get the "CREATE TABLE" statement
table_name = sanitize_string('test-{id}'.format(id=spreadsheet_id)).upper()
schema_name = 'wild_west'

# You can specify either a sheet, dataframe, or CSV file as your data source. 
# Note that create_table_stmt tries to guess the column type based on the data
sql = create_table_stmt(table_name, schema=schema_name, google_sheet=sheet)
sql_df = create_table_stmt(table_name, schema=schema_name, dataframe=df)
sql_csv = create_table_stmt(table_name, schema=schema_name, csv_filename=csv_filename)
# The above are equivalent because the data is the same

print(sql)

CREATE TABLE wild_west.TEST_1BXIMVS0XRA5NFMDKVBDBZJGMUUQPTLBS74OGVE2UPMS (class_level VARCHAR, extracurricular_activity VARCHAR, gender VARCHAR, home_state VARCHAR, major VARCHAR, student_name VARCHAR) COMMENT = ''


In [4]:
# Create the table in the Snowflake warehouse
(c, results) = Warehouse.execute(sql)
for result in results:
    print(result)

('Table TEST_1BXIMVS0XRA5NFMDKVBDBZJGMUUQPTLBS74OGVE2UPMS successfully created.',)


In [5]:
# Upload data
reflected_table = Warehouse.reflect(table_name, schema=schema_name)

# Like create_table_stmt, you can upload from a sheet, dataframe, or CSV file
# Note that upload_to_warehouse guesses the data type of each column for google_sheet and csv_filename
# If you need to force the column type, e.g., you have a number column with leading zeros that needs to be
# uploaded as a string, manually convert the Google Sheet or CSV to a dataframe before uploading.
upload_to_warehouse(reflected_table, google_sheet=sheet)
#upload_to_warehouse(reflected_table, csv_filename=csv_filename)
#upload_to_warehouse(reflected_table, dataframe=df)
#upload_to_warehouse(reflected_table, google_drive_id=google_drive_id) #See googledrive-example.ipynb for more details

# Additional parameters that you can pass to both upload_to_warehouse and create_table_stmt
# encoding='<encoding-name>': default value is 'utf-8'
# force_string=True: default value is False. This forces all columns to be interpreted as a string (good for leading zeros)
#     force_string does NOT do anything if you pass a dataframe
# batch_size=<integer>: upload_to_warehouse only; default value is 200


30 rows to insert
Inserted 30 rows to wild_west.TEST_1BXIMVS0XRA5NFMDKVBDBZJGMUUQPTLBS74OGVE2UPMS


In [6]:
# Read the data back
Warehouse.read_sql('SELECT * FROM {table} LIMIT {limit}'.format(table=table_name, limit=30))

,class_level,extracurricular_activity,gender,home_state,major,student_name
0,4. Senior,Drama Club,Female,CA,English,Alexandra
1,1. Freshman,Lacrosse,Male,SD,Math,Andrew
2,1. Freshman,Basketball,Female,NC,English,Anna
3,2. Sophomore,Baseball,Female,SD,Art,Becky
4,4. Senior,Basketball,Male,WI,English,Benjamin
5,3. Junior,Debate,Male,MD,Art,Carl
6,3. Junior,Track & Field,Female,NE,English,Carrie
7,4. Senior,Lacrosse,Female,MD,Math,Dorothy
8,1. Freshman,Baseball,Male,MA,Math,Dylan
9,3. Junior,Drama Club,Male,FL,English,Edward


In [7]:
# Delete the test table
(conn, results) = Warehouse.execute('DROP TABLE {}'.format(table_name))
for r in results:
    print(r)

('TEST_1BXIMVS0XRA5NFMDKVBDBZJGMUUQPTLBS74OGVE2UPMS successfully dropped.',)
